In [40]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer # matrix construction
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, f1_score, fbeta_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import multilabel_confusion_matrix, plot_confusion_matrix, classification_report
from sklearn.preprocessing import MultiLabelBinarizer

import pandas as pd
import json
import os
import re

import spacy
import matplotlib
import plotly.express as px
import plotly.subplots as sp
from plotly.subplots import make_subplots
from ast import literal_eval
from tqdm import tqdm

import sklearn.metrics
import numpy as np


In [3]:
data = '../src/rcatt_training_data_original.csv'

In [4]:
df = pd.read_csv(data).reset_index(drop = True)

In [5]:
df

,Text,TA0006,TA0002,TA0040,TA0003,TA0004,TA0008,TA0005,TA0010,TA0007,...,T1124,T1035,T1086,T1490,T1216,T1094,T1043,T1211,T1127,T1077
0,Talos Blog || Cisco Talos Intelligence Group -...,0,1,0,1,1,0,1,0,1,...,1,0,0,0,0,0,0,0,0,0
1,OilRig Actors Provide a Glimpse into Developme...,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Hogfish Redleaves Malware Threat Analysis I Ac...,1,1,0,1,0,0,1,0,1,...,0,0,0,0,0,0,1,0,0,0
3,Scheduled Transfer - Enterprise | MITRE ATT&CK...,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,Winlogon Helper DLL - Enterprise | MITRE ATT&C...,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1485,"Xen - Wikipedia Xen From Wikipedia, the free e...",0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1486,An Introduction to Hardware-Assisted Virtual M...,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1487,virtualization.info | Debunking Blue Pill myth...,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1488,Hypervisor - Wikipedia Hypervisor From Wikiped...,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [23]:
df.iloc[: , :1] # visualise text column

,Text
0,Talos Blog || Cisco Talos Intelligence Group -...
1,OilRig Actors Provide a Glimpse into Developme...
2,Hogfish Redleaves Malware Threat Analysis I Ac...
3,Scheduled Transfer - Enterprise | MITRE ATT&CK...
4,Winlogon Helper DLL - Enterprise | MITRE ATT&C...
...,...
1485,"Xen - Wikipedia Xen From Wikipedia, the free e..."
1486,An Introduction to Hardware-Assisted Virtual M...
1487,virtualization.info | Debunking Blue Pill myth...
1488,Hypervisor - Wikipedia Hypervisor From Wikiped...


# Feature Extraction:

In [65]:
tf_idf = TfidfVectorizer(analyzer = 'word', stop_words='english', lowercase=True, min_df = 2, max_df = 0.99)

data = tf_idf.fit_transform(df['Text'])

X = pd.DataFrame(data.toarray(), columns=tf_idf.get_feature_names()) 

/Users/liviafries/opt/anaconda3/envs/auto_cti/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [66]:
Y = df[[col for col in df.columns if col.startswith('TA')]]

# Naive Bayes: 

In [93]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.3,
                                                    random_state = 10)

naive_bayes_classifier = OneVsRestClassifier(MultinomialNB())
naive_bayes_classifier.fit(x_train, y_train)


OneVsRestClassifier(estimator=MultinomialNB())

In [99]:
y_pred_proba = pd.DataFrame(naive_bayes_classifier.predict_proba(x_train), columns=y_train.columns)

In [100]:
y_pred = (y_pred_proba > 0.02).astype(int)

In [101]:
f_score_dict = {}
for col in y_test.columns:
    print(col)
    f1_dict[col] = fbeta_score(y_train[col], y_pred[col],beta=0.5)
    print(classification_report(y_train[col], y_pred[col]))

TA0006
              precision    recall  f1-score   support

           0       0.83      0.96      0.89       820
           1       0.62      0.26      0.36       223

    accuracy                           0.81      1043
   macro avg       0.72      0.61      0.62      1043
weighted avg       0.78      0.81      0.77      1043

TA0002
              precision    recall  f1-score   support

           0       0.91      0.51      0.66       700
           1       0.47      0.90      0.62       343

    accuracy                           0.64      1043
   macro avg       0.69      0.71      0.64      1043
weighted avg       0.77      0.64      0.64      1043

TA0040
              precision    recall  f1-score   support

           0       0.95      1.00      0.97       990
           1       0.00      0.00      0.00        53

    accuracy                           0.95      1043
   macro avg       0.47      0.50      0.49      1043
weighted avg       0.90      0.95      0.92      1043

In [102]:
y_pred_proba = pd.DataFrame(naive_bayes_classifier.predict_proba(x_test), columns=y_test.columns)

In [103]:
y_pred = (y_pred_proba > 0.005).astype(int) # probability of report being part of the tactic

In [104]:
f_score_dict = {}
for col in y_test.columns:
    print(col)
    f_score_dict[col] = fbeta_score(y_test[col], y_pred[col],beta=0.5)
    print(classification_report(y_test[col], y_pred[col]))

TA0006
              precision    recall  f1-score   support

           0       0.84      0.71      0.77       364
           1       0.24      0.40      0.30        83

    accuracy                           0.66       447
   macro avg       0.54      0.56      0.54       447
weighted avg       0.73      0.66      0.68       447

TA0002
              precision    recall  f1-score   support

           0       0.90      0.27      0.42       302
           1       0.38      0.94      0.54       145

    accuracy                           0.49       447
   macro avg       0.64      0.60      0.48       447
weighted avg       0.73      0.49      0.46       447

TA0040
              precision    recall  f1-score   support

           0       0.95      1.00      0.97       425
           1       0.00      0.00      0.00        22

    accuracy                           0.95       447
   macro avg       0.48      0.50      0.49       447
weighted avg       0.90      0.95      0.92       447

In [105]:
f_score_dict

{'TA0006': 0.26148969889064977,
 'TA0002': 0.4333970681963034,
 'TA0040': 0.0,
 'TA0003': 0.4798127559976594,
 'TA0004': 0.3210678210678211,
 'TA0008': 0.30885780885780884,
 'TA0005': 0.5955603681645911,
 'TA0010': 0.15625,
 'TA0007': 0.4026503567787971,
 'TA0009': 0.17921146953405018,
 'TA0011': 0.4028436018957346,
 'TA0001': 0.1744186046511628}

In [106]:
np.mean(list(f_score_dict.values()))

0.30962996283621486

# SVM: 

In [116]:

sv_classifier = OneVsRestClassifier(LinearSVC(penalty = 'l2', loss = 'squared_hinge', dual = False, max_iter = 1000, class_weight = 'balanced'), n_jobs = 1)
sv_classifier.fit(x_train, y_train)


OneVsRestClassifier(estimator=LinearSVC(class_weight='balanced', dual=False),
                    n_jobs=1)

In [132]:
y_pred = pd.DataFrame(sv_classifier.predict(x_train), columns=y_train.columns)

In [133]:
f_score_dict = {}
for col in y_test.columns:
    print(col)
    f_score_dict[col] = fbeta_score(y_train[col], y_pred[col],beta=0.5)
    print(classification_report(y_train[col], y_pred[col]))

TA0006
              precision    recall  f1-score   support

           0       1.00      0.98      0.99       820
           1       0.93      0.99      0.96       223

    accuracy                           0.98      1043
   macro avg       0.96      0.99      0.97      1043
weighted avg       0.98      0.98      0.98      1043

TA0002
              precision    recall  f1-score   support

           0       0.99      0.97      0.98       700
           1       0.94      0.98      0.96       343

    accuracy                           0.97      1043
   macro avg       0.97      0.98      0.97      1043
weighted avg       0.97      0.97      0.97      1043

TA0040
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       990
           1       0.95      1.00      0.97        53

    accuracy                           1.00      1043
   macro avg       0.97      1.00      0.99      1043
weighted avg       1.00      1.00      1.00      1043

In [134]:
y_pred = pd.DataFrame(sv_classifier.predict(x_test), columns=y_test.columns)

In [135]:
f_score_dict = {}
for col in y_test.columns:
    print(col)
    f_score_dict[col] = fbeta_score(y_test[col], y_pred[col],beta=0.5)
    print(classification_report(y_test[col], y_pred[col]))

TA0006
              precision    recall  f1-score   support

           0       0.90      0.91      0.90       364
           1       0.57      0.55      0.56        83

    accuracy                           0.84       447
   macro avg       0.74      0.73      0.73       447
weighted avg       0.84      0.84      0.84       447

TA0002
              precision    recall  f1-score   support

           0       0.86      0.81      0.83       302
           1       0.64      0.72      0.68       145

    accuracy                           0.78       447
   macro avg       0.75      0.76      0.75       447
weighted avg       0.79      0.78      0.78       447

TA0040
              precision    recall  f1-score   support

           0       0.97      0.99      0.98       425
           1       0.70      0.32      0.44        22

    accuracy                           0.96       447
   macro avg       0.83      0.66      0.71       447
weighted avg       0.95      0.96      0.95       447

In [136]:
f_score_dict

{'TA0006': 0.5707196029776674,
 'TA0002': 0.6557377049180328,
 'TA0040': 0.564516129032258,
 'TA0003': 0.7257072570725707,
 'TA0004': 0.5212014134275619,
 'TA0008': 0.5313588850174217,
 'TA0005': 0.7542147293700089,
 'TA0010': 0.17857142857142855,
 'TA0007': 0.5990016638935108,
 'TA0009': 0.5408970976253298,
 'TA0011': 0.7028985507246377,
 'TA0001': 0.5555555555555556}

In [137]:
np.mean(list(f_score_dict.values()))

0.5750316681821653